In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
import gradio as gr
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
import math  
import sklearn.metrics

In [2]:
df = pd.read_csv('./htrain.csv')
dfh = pd.read_csv('./hr.csv')

In [3]:
dfh['is_humour']=dfh['humour'].apply(lambda x: 0 if x=='not_funny' else 1)
dfh['funny']=dfh['humour'].apply(lambda x: 1 if x=='funny' else 0)
dfh['very_funny']=dfh['humour'].apply(lambda x: 1 if x=='very_funny' else 0)
dfh['hilarious']=dfh['humour'].apply(lambda x: 1 if x=='hilarious' else 0)
dfh.head()

,Id,ocr_text,humour,is_humour,funny,very_funny,hilarious
0,1,enters into a wrong class teacher and students,funny,1,1,0,0
1,2,dj if you single make some nooooissssseeee my ...,funny,1,1,0,0
2,3,everyone sit like a lady me,funny,1,1,0,0
3,4,when youre about to watch a 2 hour educational...,funny,1,1,0,0
4,5,government work from home olympic swimmers nsdf,very_funny,1,0,1,0


In [4]:
Xh = dfh['ocr_text']
yh = dfh[dfh.columns[3:]].values
X = df['text']
y = df[df.columns[1:]].values

In [5]:
t = pd.read_csv('./ttrain.csv')
Xt = t['comment_text']

In [6]:
MAX_FEATURES = 200000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [25]:
vectorizer.adapt(Xt.values)
vectorized_texth = vectorizer(X.values)

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_texth, y))
dataset = dataset.cache()
dataset = dataset.shuffle(20002)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [28]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [10]:
model = tf.keras.models.load_model('rhumour.h5')
model1 = tf.keras.models.load_model('thumour.h5')

In [11]:
# # model.predict(["cn is computer networks"])
p= Precision()
vectorizer(["what is computer"])

<tf.Tensor: shape=(1, 1800), dtype=int64, numpy=array([[  40,    9, 1003, ...,    0,    0,    0]], dtype=int64)>

In [12]:
# # # 2. Create Sequential Model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
# model = Sequential()
# # Create the embedding layer 
# model.add(Embedding(MAX_FEATURES+1, 32))
# # Bidirectional LSTM Layer
# model.add(Bidirectional(LSTM(32, activation='tanh')))
# # Feature extractor Fully connected layers
# model.add(Dense(128, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# # Final layer
a = Recall()
# # model.add(Dense(6, activation='sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='BinaryCrossentropy', optimizer='Adam')
# model.summary()


In [13]:
# history = model.fit(train, epochs=12, validation_data=val)

In [14]:
# model.save('thumour.h5')

In [15]:
input_text = vectorizer(["Research on the Newton’s Laws of motion, and make a report that provide detail explanation"])
model1.predict(input_text)

array([[0.8372658]], dtype=float32)

In [16]:
def score_comment(comment):
    input_text = vectorizer([comment])
    res=(model.predict(input_text))
    res1=(model1.predict(input_text))
    out=''
    l=[]
    a=(res >=0.5).astype(int)
    for i in range(1,len(a[0])):
        if a[0][i]==1:
            l.append(i)
    if res1[0][0]>0.95:
        try:
            c=max(max(l),len(l))
            out += 'is_humor : true\n'
            out += '{}: {}\n'.format('humor_rate between 1 to 3', c)
        except:
            out += 'is_humor : False\n'
    else:
        out += 'is_humor : False\n'
    return(out)

In [17]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\srini\anaconda3\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


IMPORTANT: You are using gradio version 3.11.0, however version 3.14.0 is available, please upgrade.
--------


In [18]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://11d22baa971b7fca.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [19]:
r=Recall()

In [29]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model1.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    a.update_state(y_true, yhat)
    p.update_state(y_true, yhat)
    r.update_state(y_true, yhat)
    mse = sklearn.metrics.mean_squared_error(y_true, yhat)
    rmse = math.sqrt(mse)  

In [30]:
acc=a.result().numpy()
rec=r.result().numpy()
pre=p.result().numpy()
f1=2*((p.result().numpy()*r.result().numpy())/(p.result().numpy()+r.result().numpy()))

In [32]:
# for Detecting and rating humour
print(f'Accuracy:{acc}\n\nPrecision:{pre}\n\nRecall:{rec}\n\nF1Score:{f1}\n\nRMSE:{rmse}')

Accuracy:0.981632649898529

Precision:0.991752564907074

Recall:0.971632649898529

F1Score:0.9866666197776794

RMSE:0.10321832137843548


Socket exception: An existing connection was forcibly closed by the remote host (10054)
